# Sorting Algorithms

*Organizing data efficiently for search and analysis*


## 🎯 Learning Objectives

- Understand the fundamental sorting problem and comparison-based sorting lower bound of Ω(n log n)
- Implement elementary sorts (bubble, selection, insertion) and understand when they're appropriate
- Implement efficient divide-and-conquer sorts: merge sort (stable, O(n log n)) and quick sort (in-place, average O(n log n))
- Implement heap sort and understand non-comparison sorts (counting sort, radix sort) that break the Ω(n log n) barrier
- Use Python's built-in sorting effectively with custom keys and understand Timsort's hybrid approach

---
## 1. Introduction to Sorting


Sorting is one of the most fundamental operations in computer science. It transforms an unordered collection into an ordered one, enabling efficient searching, duplicate detection, and data analysis. The choice of sorting algorithm depends on data size, memory constraints, and whether the data is nearly sorted.

> 📖 **Definition:** A **sorting algorithm** rearranges elements of a collection into a specific order (typically ascending or descending). Key properties include: **stability** (preserves relative order of equal elements), **in-place** (uses O(1) extra space), and **adaptivity** (faster on partially sorted input).

### Sorting Algorithm Comparison

| Algorithm | Best | Average | Worst | Space | Stable |
| --- | --- | --- | --- | --- | --- |
| Bubble Sort | O(n) | O(n²) | O(n²) | O(1) | Yes |
| Selection Sort | O(n²) | O(n²) | O(n²) | O(1) | No |
| Insertion Sort | O(n) | O(n²) | O(n²) | O(1) | Yes |
| Merge Sort | O(n log n) | O(n log n) | O(n log n) | O(n) | Yes |
| Quick Sort | O(n log n) | O(n log n) | O(n²) | O(log n) | No |
| Heap Sort | O(n log n) | O(n log n) | O(n log n) | O(1) | No |

**Listing 10.1 — Sorting Performance Comparison**

In [ ]:
import time
import random

def measure_sort(sort_func, arr):
    """Measure sorting time."""
    arr_copy = arr.copy()
    start = time.perf_counter()
    sort_func(arr_copy)
    return time.perf_counter() - start

# Simple O(n²) sorts for comparison
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

def insertion_sort(arr):
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j+1] = arr[j]
            j -= 1
        arr[j+1] = key

# Test with small array (O(n²) is slow!)
n = 1000
arr = [random.randint(0, 10000) for _ in range(n)]

print(f"Sorting {n} elements:")
print(f"  Bubble sort:    {measure_sort(bubble_sort, arr)*1000:.2f} ms")
print(f"  Insertion sort: {measure_sort(insertion_sort, arr)*1000:.2f} ms")
print(f"  Python sorted:  {measure_sort(sorted, arr)*1000:.4f} ms")
print(f"\nPython's Timsort is ~100x faster!")

***Figure 10.1:** O(n²) algorithms are dramatically slower than O(n log n) for large inputs.*

---
## 2. Bubble Sort


> 📖 **Definition:** **Bubble Sort** repeatedly steps through the list, compares adjacent elements, and swaps them if they're in wrong order. The largest element "bubbles up" to its correct position each pass. Simple but inefficient—rarely used in practice.

**Listing 10.2 — Bubble Sort Basic**

In [ ]:
def bubble_sort(arr):
    """
    Bubble Sort - O(n²) time, O(1) space, stable.
    """
    n = len(arr)
    
    for i in range(n):
        # Last i elements are already sorted
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    
    return arr

# Trace execution
arr = [64, 34, 25, 12, 22, 11, 90]
print(f"Original: {arr}")

n = len(arr)
for i in range(n):
    for j in range(0, n - i - 1):
        if arr[j] > arr[j + 1]:
            arr[j], arr[j + 1] = arr[j + 1], arr[j]
    print(f"Pass {i+1}:   {arr}  (largest {arr[n-i-1]} in place)")

***Figure 10.2:** Each pass moves the largest unsorted element to its final position.*

**Listing 10.3 — Optimized Bubble Sort**

In [ ]:
def bubble_sort_optimized(arr):
    """
    Optimized Bubble Sort - stops early if already sorted.
    Best case O(n) for sorted input.
    """
    n = len(arr)
    
    for i in range(n):
        swapped = False
        
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        
        # If no swaps, array is sorted
        if not swapped:
            print(f"  Early termination at pass {i+1}")
            break
    
    return arr

# Test on nearly sorted array
nearly_sorted = [1, 2, 3, 5, 4, 6, 7, 8]
print(f"Nearly sorted: {nearly_sorted}")
result = bubble_sort_optimized(nearly_sorted.copy())
print(f"Sorted: {result}")

print("\nAlready sorted array:")
sorted_arr = [1, 2, 3, 4, 5]
bubble_sort_optimized(sorted_arr.copy())

***Figure 10.3:** Early termination optimization makes bubble sort O(n) for already-sorted input.*

---
## 3. Selection Sort


> 📖 **Definition:** **Selection Sort** divides the array into sorted and unsorted portions. It repeatedly finds the minimum element from the unsorted portion and moves it to the end of the sorted portion. Makes minimum number of swaps (n-1).

**Listing 10.4 — Selection Sort**

In [ ]:
def selection_sort(arr):
    """
    Selection Sort - O(n²) time, O(1) space, NOT stable.
    """
    n = len(arr)
    
    for i in range(n):
        # Find minimum in unsorted portion
        min_idx = i
        for j in range(i + 1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        
        # Swap minimum with first unsorted element
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    
    return arr

# Trace execution
arr = [64, 25, 12, 22, 11]
print(f"Original: {arr}")

n = len(arr)
for i in range(n):
    min_idx = i
    for j in range(i + 1, n):
        if arr[j] < arr[min_idx]:
            min_idx = j
    arr[i], arr[min_idx] = arr[min_idx], arr[i]
    print(f"Pass {i+1}: {arr}  (min {arr[i]} placed at index {i})")

***Figure 10.4:** Selection sort makes exactly n-1 swaps regardless of input order.*

---
## 4. Insertion Sort


> 📖 **Definition:** **Insertion Sort** builds the sorted array one element at a time by inserting each element into its correct position among the previously sorted elements. Like sorting cards in your hand—efficient for small or nearly sorted data.

**Listing 10.5 — Insertion Sort**

In [ ]:
def insertion_sort(arr):
    """
    Insertion Sort - O(n²) average, O(n) best, O(1) space, stable.
    """
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        
        # Shift elements greater than key
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        
        # Insert key at correct position
        arr[j + 1] = key
    
    return arr

# Trace execution
arr = [12, 11, 13, 5, 6]
print(f"Original: {arr}")

for i in range(1, len(arr)):
    key = arr[i]
    j = i - 1
    while j >= 0 and arr[j] > key:
        arr[j + 1] = arr[j]
        j -= 1
    arr[j + 1] = key
    print(f"Insert {key}: {arr}  (sorted: {arr[:i+1]})")

***Figure 10.5:** Insertion sort maintains a sorted portion and inserts each new element into it.*

**Listing 10.6 — Binary Insertion Sort**

In [ ]:
import bisect

def binary_insertion_sort(arr):
    """
    Binary Insertion Sort - O(n log n) comparisons, O(n²) shifts.
    Uses binary search to find insertion position.
    """
    for i in range(1, len(arr)):
        key = arr[i]
        # Binary search for insertion point
        pos = bisect.bisect_left(arr, key, 0, i)
        # Shift elements and insert
        arr[pos+1:i+1] = arr[pos:i]
        arr[pos] = key
    
    return arr

# Compare comparison counts
def count_comparisons(arr):
    """Count comparisons in standard insertion sort."""
    comparisons = 0
    arr = arr.copy()
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            comparisons += 1
            arr[j + 1] = arr[j]
            j -= 1
        if j >= 0:
            comparisons += 1
        arr[j + 1] = key
    return comparisons

import random
arr = [random.randint(0, 100) for _ in range(20)]
print(f"Standard insertion: ~{count_comparisons(arr)} comparisons")
print(f"Binary insertion: ~{20 * 4} comparisons (n * log n)")
print(f"\nNote: Binary search reduces comparisons but not shifts")

***Figure 10.6:** Binary search reduces comparisons but shifting elements still takes O(n) per insertion.*

---
## 5. Merge Sort


> 📖 **Definition:** **Merge Sort** is a divide-and-conquer algorithm that splits the array in half, recursively sorts each half, then merges the sorted halves. Guaranteed O(n log n) but requires O(n) extra space. Stable and parallelizable.

**Listing 10.7 — Merge Sort**

In [ ]:
def merge_sort(arr):
    """
    Merge Sort - O(n log n) time, O(n) space, stable.
    """
    if len(arr) <= 1:
        return arr
    
    # Divide
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    # Conquer (merge)
    return merge(left, right)

def merge(left, right):
    """Merge two sorted arrays."""
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:  # <= for stability
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    # Append remaining elements
    result.extend(left[i:])
    result.extend(right[j:])
    return result

# Test
arr = [38, 27, 43, 3, 9, 82, 10]
print(f"Original: {arr}")
print(f"Sorted:   {merge_sort(arr)}")

***Figure 10.7:** Merge sort recursively divides, then merges sorted halves in O(n) time.*

**Listing 10.8 — Merge Sort In-Place**

In [ ]:
def merge_sort_inplace(arr, left=0, right=None):
    """
    In-place merge sort (still uses O(n) temp space for merge).
    """
    if right is None:
        right = len(arr) - 1
    
    if left < right:
        mid = (left + right) // 2
        merge_sort_inplace(arr, left, mid)
        merge_sort_inplace(arr, mid + 1, right)
        merge_inplace(arr, left, mid, right)

def merge_inplace(arr, left, mid, right):
    """Merge arr[left:mid+1] and arr[mid+1:right+1]."""
    # Create temp arrays
    left_arr = arr[left:mid + 1]
    right_arr = arr[mid + 1:right + 1]
    
    i = j = 0
    k = left
    
    while i < len(left_arr) and j < len(right_arr):
        if left_arr[i] <= right_arr[j]:
            arr[k] = left_arr[i]
            i += 1
        else:
            arr[k] = right_arr[j]
            j += 1
        k += 1
    
    while i < len(left_arr):
        arr[k] = left_arr[i]
        i += 1
        k += 1
    
    while j < len(right_arr):
        arr[k] = right_arr[j]
        j += 1
        k += 1

# Test
arr = [12, 11, 13, 5, 6, 7]
print(f"Original: {arr}")
merge_sort_inplace(arr)
print(f"Sorted:   {arr}")

***Figure 10.8:** In-place merge sort modifies the original array but still needs O(n) temp space for merging.*

**Listing 10.9 — Bottom-Up Merge Sort**

In [ ]:
def merge_sort_bottom_up(arr):
    """
    Iterative (bottom-up) merge sort.
    Avoids recursion overhead, same O(n log n) complexity.
    """
    n = len(arr)
    
    # Start with size 1, double each iteration
    size = 1
    while size < n:
        for left in range(0, n, 2 * size):
            mid = min(left + size - 1, n - 1)
            right = min(left + 2 * size - 1, n - 1)
            
            if mid < right:
                merge_inplace(arr, left, mid, right)
        
        print(f"Size {size}: {arr}")
        size *= 2
    
    return arr

def merge_inplace(arr, left, mid, right):
    left_arr = arr[left:mid + 1]
    right_arr = arr[mid + 1:right + 1]
    i = j = 0
    k = left
    while i < len(left_arr) and j < len(right_arr):
        if left_arr[i] <= right_arr[j]:
            arr[k] = left_arr[i]
            i += 1
        else:
            arr[k] = right_arr[j]
            j += 1
        k += 1
    while i < len(left_arr):
        arr[k] = left_arr[i]
        i += 1
        k += 1
    while j < len(right_arr):
        arr[k] = right_arr[j]
        j += 1
        k += 1

# Test
arr = [64, 34, 25, 12, 22, 11, 90, 8]
print(f"Original: {arr}\n")
merge_sort_bottom_up(arr)

***Figure 10.9:** Bottom-up merge sort merges pairs of size 1, then 2, then 4, etc.*

---
## 6. Quick Sort


> 📖 **Definition:** **Quick Sort** selects a "pivot" element and partitions the array so elements less than pivot come before it, and elements greater come after. Recursively sorts the partitions. Average O(n log n), worst O(n²), but fastest in practice for most inputs.

**Listing 10.10 — Quick Sort Basic**

In [ ]:
def quick_sort(arr):
    """
    Quick Sort - O(n log n) average, O(n²) worst, O(log n) space.
    Simple implementation (not in-place).
    """
    if len(arr) <= 1:
        return arr
    
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    
    return quick_sort(left) + middle + quick_sort(right)

# Test
arr = [3, 6, 8, 10, 1, 2, 1]
print(f"Original: {arr}")
print(f"Sorted:   {quick_sort(arr)}")

***Figure 10.10:** Simple quick sort creates new lists for partitions (not in-place but easy to understand).*

**Listing 10.11 — Quick Sort In-Place (Lomuto)**

In [ ]:
def quick_sort_inplace(arr, low=0, high=None):
    """In-place quick sort using Lomuto partition."""
    if high is None:
        high = len(arr) - 1
    
    if low < high:
        pivot_idx = partition_lomuto(arr, low, high)
        quick_sort_inplace(arr, low, pivot_idx - 1)
        quick_sort_inplace(arr, pivot_idx + 1, high)

def partition_lomuto(arr, low, high):
    """
    Lomuto partition scheme.
    Pivot is last element, returns final pivot position.
    """
    pivot = arr[high]
    i = low - 1  # Index of smaller element
    
    for j in range(low, high):
        if arr[j] <= pivot:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    
    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1

# Trace partition
arr = [10, 7, 8, 9, 1, 5]
print(f"Original: {arr}")
print(f"Pivot (last element): {arr[-1]}")

pivot_idx = partition_lomuto(arr, 0, len(arr) - 1)
print(f"After partition: {arr}")
print(f"Pivot index: {pivot_idx} (all left < {arr[pivot_idx]} < all right)")

***Figure 10.11:** Lomuto partition uses last element as pivot and maintains a boundary of smaller elements.*

**Listing 10.12 — Quick Sort (Hoare Partition)**

In [ ]:
def partition_hoare(arr, low, high):
    """
    Hoare partition scheme - more efficient than Lomuto.
    Uses two pointers moving towards each other.
    """
    pivot = arr[(low + high) // 2]
    i = low - 1
    j = high + 1
    
    while True:
        i += 1
        while arr[i] < pivot:
            i += 1
        
        j -= 1
        while arr[j] > pivot:
            j -= 1
        
        if i >= j:
            return j
        
        arr[i], arr[j] = arr[j], arr[i]

def quick_sort_hoare(arr, low=0, high=None):
    """Quick sort using Hoare partition."""
    if high is None:
        high = len(arr) - 1
    
    if low < high:
        p = partition_hoare(arr, low, high)
        quick_sort_hoare(arr, low, p)
        quick_sort_hoare(arr, p + 1, high)

# Test
arr = [8, 3, 1, 7, 0, 10, 2]
print(f"Original: {arr}")
quick_sort_hoare(arr)
print(f"Sorted:   {arr}")

***Figure 10.12:** Hoare partition uses middle pivot and two pointers, making ~3x fewer swaps than Lomuto.*

**Listing 10.13 — Quick Sort with Random Pivot**

In [ ]:
import random

def quick_sort_random(arr, low=0, high=None):
    """Quick sort with random pivot to avoid worst case."""
    if high is None:
        high = len(arr) - 1
    
    if low < high:
        pivot_idx = partition_random(arr, low, high)
        quick_sort_random(arr, low, pivot_idx - 1)
        quick_sort_random(arr, pivot_idx + 1, high)

def partition_random(arr, low, high):
    """Randomly select pivot, then use Lomuto."""
    rand_idx = random.randint(low, high)
    arr[rand_idx], arr[high] = arr[high], arr[rand_idx]
    
    pivot = arr[high]
    i = low - 1
    
    for j in range(low, high):
        if arr[j] <= pivot:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    
    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1

# Test on sorted array (worst case for basic quick sort)
arr = list(range(10))  # Already sorted
print(f"Sorted input (worst case): {arr}")
quick_sort_random(arr)
print(f"Still works: {arr}")

# Many duplicates
arr = [3, 3, 3, 1, 1, 2, 2, 2]
quick_sort_random(arr)
print(f"With duplicates: {arr}")

***Figure 10.13:** Random pivot selection prevents O(n²) worst case on sorted or nearly-sorted input.*

---
## 7. Heap Sort


> 📖 **Definition:** **Heap Sort** builds a max-heap from the array, then repeatedly extracts the maximum element. Guaranteed O(n log n) time, O(1) space, but not stable and has poor cache performance compared to quick sort.

**Listing 10.14 — Heap Sort**

In [ ]:
def heap_sort(arr):
    """
    Heap Sort - O(n log n) time, O(1) space, NOT stable.
    """
    n = len(arr)
    
    # Build max heap
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
    
    # Extract elements one by one
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]  # Move max to end
        heapify(arr, i, 0)  # Restore heap property
    
    return arr

def heapify(arr, n, i):
    """Maintain max-heap property for subtree rooted at i."""
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    if left < n and arr[left] > arr[largest]:
        largest = left
    
    if right < n and arr[right] > arr[largest]:
        largest = right
    
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

# Test
arr = [12, 11, 13, 5, 6, 7]
print(f"Original: {arr}")
heap_sort(arr)
print(f"Sorted:   {arr}")

***Figure 10.14:** Heap sort builds max-heap in O(n), then extracts max n times for O(n log n) total.*

**Listing 10.15 — Heap Sort with heapq**

In [ ]:
import heapq

def heap_sort_heapq(arr):
    """Heap sort using Python's heapq (min-heap)."""
    heapq.heapify(arr)  # O(n)
    return [heapq.heappop(arr) for _ in range(len(arr))]  # O(n log n)

# Using nsmallest (even simpler)
def heap_sort_simple(arr):
    """Simplest heap sort using nsmallest."""
    return heapq.nsmallest(len(arr), arr)

arr = [3, 1, 4, 1, 5, 9, 2, 6]
print(f"Original: {arr}")
print(f"Using heapq: {heap_sort_heapq(arr.copy())}")
print(f"Using nsmallest: {heap_sort_simple(arr.copy())}")

***Figure 10.15:** Python's heapq provides simple heap sort, though it's not in-place.*

---
## 8. Counting & Radix Sort


> 📖 **Definition:** **Counting Sort** counts occurrences of each value and uses cumulative counts to place elements. O(n + k) where k is the range of values. **Radix Sort** sorts by each digit using counting sort as a subroutine, achieving O(d(n + k)) for d digits.

**Listing 10.16 — Counting Sort**

In [ ]:
def counting_sort(arr, max_val=None):
    """
    Counting Sort - O(n + k) time, O(k) space, stable.
    Only for non-negative integers with known range.
    """
    if not arr:
        return arr
    
    if max_val is None:
        max_val = max(arr)
    
    # Count occurrences
    count = [0] * (max_val + 1)
    for num in arr:
        count[num] += 1
    
    # Cumulative count (position of each element)
    for i in range(1, len(count)):
        count[i] += count[i - 1]
    
    # Build output array (traverse in reverse for stability)
    output = [0] * len(arr)
    for num in reversed(arr):
        output[count[num] - 1] = num
        count[num] -= 1
    
    return output

# Test
arr = [4, 2, 2, 8, 3, 3, 1]
print(f"Original: {arr}")
print(f"Sorted:   {counting_sort(arr)}")

# When it's efficient
arr = [1, 4, 1, 2, 7, 5, 2]  # Range 1-7, n=7
print(f"\nSmall range (1-7): Counting sort is O(n)")

***Figure 10.16:** Counting sort is O(n) when the range of values is O(n).*

**Listing 10.17 — Radix Sort**

In [ ]:
def radix_sort(arr):
    """
    Radix Sort - O(d * n) time where d = number of digits.
    Uses counting sort for each digit position.
    """
    if not arr:
        return arr
    
    max_val = max(arr)
    exp = 1  # Current digit position (1, 10, 100, ...)
    
    while max_val // exp > 0:
        counting_sort_by_digit(arr, exp)
        exp *= 10
    
    return arr

def counting_sort_by_digit(arr, exp):
    """Counting sort based on digit at exp position."""
    n = len(arr)
    output = [0] * n
    count = [0] * 10
    
    # Count occurrences of digits
    for num in arr:
        digit = (num // exp) % 10
        count[digit] += 1
    
    # Cumulative count
    for i in range(1, 10):
        count[i] += count[i - 1]
    
    # Build output (reverse for stability)
    for i in range(n - 1, -1, -1):
        digit = (arr[i] // exp) % 10
        output[count[digit] - 1] = arr[i]
        count[digit] -= 1
    
    # Copy back
    for i in range(n):
        arr[i] = output[i]

# Test
arr = [170, 45, 75, 90, 802, 24, 2, 66]
print(f"Original: {arr}")

# Trace each pass
arr_copy = arr.copy()
max_val = max(arr_copy)
exp = 1
while max_val // exp > 0:
    counting_sort_by_digit(arr_copy, exp)
    print(f"After sorting by {exp}'s place: {arr_copy}")
    exp *= 10

***Figure 10.17:** Radix sort processes digits from least significant to most significant.*

---
## 9. Python's Built-in Sorting


**Listing 10.18 — sorted() vs list.sort()**

In [ ]:
# sorted() - returns new list, original unchanged
arr = [3, 1, 4, 1, 5, 9, 2, 6]
sorted_arr = sorted(arr)
print(f"Original after sorted(): {arr}")
print(f"New sorted list: {sorted_arr}")

# list.sort() - modifies in place, returns None
arr = [3, 1, 4, 1, 5, 9, 2, 6]
result = arr.sort()
print(f"\nOriginal after .sort(): {arr}")
print(f"Return value: {result}")  # None!

# Reverse order
arr = [3, 1, 4, 1, 5]
print(f"\nDescending: {sorted(arr, reverse=True)}")

***Figure 10.18:** sorted() creates new list; list.sort() modifies in-place and returns None.*

**Listing 10.19 — Custom Sort Keys**

In [ ]:
# Sort by custom key function
words = ["banana", "pie", "apple", "cherry"]

# By length
print(f"By length: {sorted(words, key=len)}")

# By last character
print(f"By last char: {sorted(words, key=lambda x: x[-1])}")

# Multiple criteria: length, then alphabetically
print(f"By (length, alpha): {sorted(words, key=lambda x: (len(x), x))}")

# Sort objects
students = [
    {"name": "Alice", "grade": 85},
    {"name": "Bob", "grade": 90},
    {"name": "Charlie", "grade": 85}
]

# By grade (descending), then name
sorted_students = sorted(students, key=lambda s: (-s["grade"], s["name"]))
print(f"\nBy grade desc, name asc:")
for s in sorted_students:
    print(f"  {s['name']}: {s['grade']}")

***Figure 10.19:** The key parameter extracts comparison keys. Use tuples for multi-field sorting.*

**Listing 10.20 — Timsort Characteristics**

In [ ]:
import time
import random

def measure(arr, desc):
    start = time.perf_counter()
    sorted(arr)
    elapsed = time.perf_counter() - start
    print(f"  {desc}: {elapsed*1000:.2f} ms")

n = 100000

# Timsort adapts to input patterns
print(f"Sorting {n} elements:")

# Random - O(n log n)
measure([random.randint(0, n) for _ in range(n)], "Random")

# Already sorted - O(n) !
measure(list(range(n)), "Sorted")

# Reverse sorted - O(n) !
measure(list(range(n, 0, -1)), "Reverse")

# Few unique values
measure([random.randint(0, 10) for _ in range(n)], "Few unique")

print("\nTimsort is adaptive and exploits existing order!")

***Figure 10.20:** Timsort detects sorted runs and merges them, achieving O(n) on sorted input.*

**Listing 10.21 — Stable Sort Property**

In [ ]:
# Python's sort is stable - preserves relative order of equal elements

data = [
    ("Alice", 3),
    ("Bob", 1),
    ("Charlie", 3),
    ("David", 2),
    ("Eve", 1)
]

# Sort by score (second element)
by_score = sorted(data, key=lambda x: x[1])
print("Sorted by score (stable):")
for name, score in by_score:
    print(f"  {name}: {score}")

print("\nNotice: Alice comes before Charlie (both score 3)")
print("Bob comes before Eve (both score 1)")
print("This is the original relative order - stability!")

# Multi-level sort using stability
# Sort by name first, then by score
by_name = sorted(data, key=lambda x: x[0])
by_score_then_name = sorted(by_name, key=lambda x: x[1])
print("\nBy score, then name (via two stable sorts):")
for name, score in by_score_then_name:
    print(f"  {name}: {score}")

***Figure 10.21:** Stable sorts maintain relative order of equal elements, enabling multi-pass sorting.*

---
## 10. Common Patterns


### Pattern 1: Sort then Search

**Listing 10.22 — Two Sum with Sorting**

In [ ]:
def two_sum_sorted(nums, target):
    """
    Find pair summing to target using sorting + two pointers.
    Time: O(n log n), Space: O(1) if we can sort in-place.
    """
    nums_sorted = sorted(enumerate(nums), key=lambda x: x[1])
    left, right = 0, len(nums) - 1
    
    while left < right:
        current_sum = nums_sorted[left][1] + nums_sorted[right][1]
        
        if current_sum == target:
            return [nums_sorted[left][0], nums_sorted[right][0]]
        elif current_sum < target:
            left += 1
        else:
            right -= 1
    
    return []

# Test
nums = [2, 7, 11, 15]
target = 9
print(f"nums = {nums}, target = {target}")
print(f"Indices: {two_sum_sorted(nums, target)}")

***Figure 10.22:** Sorting enables two-pointer technique for pair finding.*

### Pattern 2: Sort by Custom Criteria

**Listing 10.23 — Largest Number**

In [ ]:
from functools import cmp_to_key

def largest_number(nums):
    """
    Arrange numbers to form largest possible number.
    Custom comparator: compare "ab" vs "ba".
    """
    def compare(a, b):
        if a + b > b + a:
            return -1
        elif a + b < b + a:
            return 1
        return 0
    
    # Convert to strings and sort with custom comparator
    nums_str = [str(n) for n in nums]
    nums_str.sort(key=cmp_to_key(compare))
    
    # Handle edge case of all zeros
    if nums_str[0] == "0":
        return "0"
    
    return "".join(nums_str)

# Test
nums = [3, 30, 34, 5, 9]
print(f"nums = {nums}")
print(f"Largest number: {largest_number(nums)}")

nums = [10, 2]
print(f"\nnums = {nums}")
print(f"Largest number: {largest_number(nums)}")

***Figure 10.23:** Custom comparator determines ordering based on concatenation result.*

### Pattern 3: Interval Problems

**Listing 10.24 — Merge Intervals**

In [ ]:
def merge_intervals(intervals):
    """
    Merge overlapping intervals.
    Key insight: sort by start, then scan.
    """
    if not intervals:
        return []
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    merged = [intervals[0]]
    
    for start, end in intervals[1:]:
        last_end = merged[-1][1]
        
        if start <= last_end:  # Overlapping
            merged[-1][1] = max(last_end, end)
        else:
            merged.append([start, end])
    
    return merged

# Test
intervals = [[1, 3], [2, 6], [8, 10], [15, 18]]
print(f"Original: {intervals}")
print(f"Merged:   {merge_intervals(intervals)}")

***Figure 10.24:** Sorting by start time simplifies interval merging to a single pass.*

### Pattern 4: Meeting Rooms

**Listing 10.25 — Meeting Rooms II**

In [ ]:
import heapq

def min_meeting_rooms(intervals):
    """
    Find minimum meeting rooms needed.
    Track end times with min-heap.
    """
    if not intervals:
        return 0
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    # Min-heap of end times
    heap = []
    
    for start, end in intervals:
        # If earliest ending room is free, reuse it
        if heap and heap[0] <= start:
            heapq.heappop(heap)
        
        heapq.heappush(heap, end)
    
    return len(heap)

# Test
intervals = [[0, 30], [5, 10], [15, 20]]
print(f"Meetings: {intervals}")
print(f"Rooms needed: {min_meeting_rooms(intervals)}")

intervals = [[7, 10], [2, 4]]
print(f"\nMeetings: {intervals}")
print(f"Rooms needed: {min_meeting_rooms(intervals)}")

***Figure 10.25:** Sort by start, use heap to track room availability by end times.*

### Pattern 5: Kth Element

**Listing 10.26 — Kth Largest (Quick Select)**

In [ ]:
import random

def find_kth_largest(nums, k):
    """
    Find kth largest element using Quick Select.
    Average O(n), worst O(n²).
    """
    k = len(nums) - k  # Convert to kth smallest
    
    def quick_select(left, right):
        pivot_idx = partition(left, right)
        
        if pivot_idx == k:
            return nums[k]
        elif pivot_idx < k:
            return quick_select(pivot_idx + 1, right)
        else:
            return quick_select(left, pivot_idx - 1)
    
    def partition(left, right):
        # Random pivot
        rand_idx = random.randint(left, right)
        nums[rand_idx], nums[right] = nums[right], nums[rand_idx]
        
        pivot = nums[right]
        i = left
        
        for j in range(left, right):
            if nums[j] <= pivot:
                nums[i], nums[j] = nums[j], nums[i]
                i += 1
        
        nums[i], nums[right] = nums[right], nums[i]
        return i
    
    return quick_select(0, len(nums) - 1)

# Test
nums = [3, 2, 1, 5, 6, 4]
k = 2
print(f"nums = {nums}, k = {k}")
print(f"{k}th largest: {find_kth_largest(nums.copy(), k)}")

# Compare with sorted
print(f"Verification: {sorted(nums, reverse=True)[k-1]}")

***Figure 10.26:** Quick Select finds kth element in O(n) average without fully sorting.*

### Pattern 6: Sort and Greedy

**Listing 10.27 — Non-overlapping Intervals**

In [ ]:
def erase_overlap_intervals(intervals):
    """
    Find minimum intervals to remove for non-overlapping set.
    Greedy: sort by end time, keep intervals that end earliest.
    """
    if not intervals:
        return 0
    
    # Sort by end time
    intervals.sort(key=lambda x: x[1])
    
    removals = 0
    prev_end = intervals[0][1]
    
    for i in range(1, len(intervals)):
        start, end = intervals[i]
        
        if start < prev_end:  # Overlap
            removals += 1
            # Keep interval with earlier end (already have it)
        else:
            prev_end = end
    
    return removals

# Test
intervals = [[1, 2], [2, 3], [3, 4], [1, 3]]
print(f"Intervals: {intervals}")
print(f"Remove {erase_overlap_intervals(intervals)} for non-overlapping")

intervals = [[1, 2], [1, 2], [1, 2]]
print(f"\nIntervals: {intervals}")
print(f"Remove {erase_overlap_intervals(intervals)} for non-overlapping")

***Figure 10.27:** Sort by end time for interval scheduling—greedy choice keeps earliest-ending intervals.*

### Pattern 7: Bucket Sort for Special Cases

**Listing 10.28 — Maximum Gap**

In [ ]:
def maximum_gap(nums):
    """
    Find maximum gap between successive elements in sorted form.
    Must be O(n) time - use bucket sort idea.
    """
    if len(nums) < 2:
        return 0
    
    min_val, max_val = min(nums), max(nums)
    n = len(nums)
    
    if min_val == max_val:
        return 0
    
    # Bucket size ensures max gap is between buckets
    bucket_size = max(1, (max_val - min_val) // (n - 1))
    bucket_count = (max_val - min_val) // bucket_size + 1
    
    # Each bucket stores (min, max) of elements in it
    buckets = [[float('inf'), float('-inf')] for _ in range(bucket_count)]
    
    for num in nums:
        idx = (num - min_val) // bucket_size
        buckets[idx][0] = min(buckets[idx][0], num)
        buckets[idx][1] = max(buckets[idx][1], num)
    
    # Max gap is between adjacent non-empty buckets
    max_gap = 0
    prev_max = min_val
    
    for bucket_min, bucket_max in buckets:
        if bucket_min == float('inf'):  # Empty bucket
            continue
        max_gap = max(max_gap, bucket_min - prev_max)
        prev_max = bucket_max
    
    return max_gap

# Test
nums = [3, 6, 9, 1]
print(f"nums = {nums}")
print(f"Sorted: {sorted(nums)}")
print(f"Maximum gap: {maximum_gap(nums)}")  # 3 (between 6 and 9)

***Figure 10.28:** Bucket sort achieves O(n) for maximum gap by exploiting the pigeonhole principle.*

- **O(n²) sorts:** Bubble, selection, insertion—simple but slow for large data
- **O(n log n) sorts:** Merge (stable, extra space), Quick (fastest avg), Heap (in-place)
- **Linear sorts:** Counting, Radix—for integers with limited range
- **Python's Timsort:** Hybrid merge+insertion, adaptive, stable, use it!
- **Custom keys:** Use lambda or key function for complex sorting

---
## 11. Common Pitfalls


### Pitfall 1: Forgetting sort() Returns None

**Listing 10.29 — sort() vs sorted()**

In [ ]:
# WRONG: list.sort() returns None!
arr = [3, 1, 2]
result = arr.sort()
print(f"result = arr.sort(): {result}")  # None!

# RIGHT: Use sorted() if you need the result
arr = [3, 1, 2]
result = sorted(arr)
print(f"result = sorted(arr): {result}")

# Or call sort() separately
arr = [3, 1, 2]
arr.sort()
print(f"After arr.sort(): {arr}")

***Figure 10.29:** list.sort() modifies in-place and returns None. Use sorted() for chaining.*

### Pitfall 2: Unstable Sort Breaking Ties

**Listing 10.30 — Stability Matters**

In [ ]:
# When stability matters: preserve original order for ties

# Example: Sort students by score, keep alphabetical for ties
students = [
    ("Alice", 85),
    ("Bob", 90),
    ("Charlie", 85),
    ("David", 85)
]

# Python's sort IS stable
by_score = sorted(students, key=lambda x: x[1])
print("Stable sort (preserves A, C, D order for score 85):")
for name, score in by_score:
    print(f"  {name}: {score}")

# Quick sort is NOT stable - would mix up Alice, Charlie, David
print("\nIf using unstable sort, explicitly add secondary key:")
by_score_name = sorted(students, key=lambda x: (x[1], x[0]))
for name, score in by_score_name:
    print(f"  {name}: {score}")

***Figure 10.30:** Use stable sort or add secondary keys to maintain deterministic ordering.*

### Pitfall 3: Wrong Pivot Selection

**Listing 10.31 — Quick Sort Worst Case**

In [ ]:
import sys
sys.setrecursionlimit(2000)

def quick_sort_bad_pivot(arr, low, high, depth=0):
    """Quick sort with first element as pivot - worst case on sorted."""
    if low < high:
        pivot_idx = partition_first(arr, low, high)
        quick_sort_bad_pivot(arr, low, pivot_idx - 1, depth + 1)
        quick_sort_bad_pivot(arr, pivot_idx + 1, high, depth + 1)
    return depth

def partition_first(arr, low, high):
    pivot = arr[low]  # BAD: first element
    i = high + 1
    for j in range(high, low, -1):
        if arr[j] >= pivot:
            i -= 1
            arr[i], arr[j] = arr[j], arr[i]
    arr[i - 1], arr[low] = arr[low], arr[i - 1]
    return i - 1

# On sorted input, recursion depth = n (worst case!)
n = 100
arr = list(range(n))
print(f"Sorting {n} already-sorted elements:")
print(f"Bad pivot (first element): O(n²) partitions")
print(f"Random pivot: O(n log n) partitions")
print(f"\nAlways use random or median-of-three pivot selection!")

***Figure 10.31:** First/last element pivot degrades to O(n²) on sorted input. Use random pivot.*

### Pitfall 4: Modifying While Sorting

**Listing 10.32 — Key Function Side Effects**

In [ ]:
# WRONG: Key function with side effects
call_count = 0

def bad_key(x):
    global call_count
    call_count += 1
    return x

arr = [3, 1, 4, 1, 5]
sorted(arr, key=bad_key)
print(f"Key function called {call_count} times for {len(arr)} elements")
print("Key is called multiple times per element during sorting!")

# WRONG: Key function that modifies data
data = [{"val": 3}, {"val": 1}, {"val": 2}]
def mutating_key(d):
    d["val"] *= 2  # DON'T DO THIS!
    return d["val"]

# This corrupts the data
# sorted(data, key=mutating_key)  # Don't run this!

print("\nKey functions should be:")
print("  - Pure (no side effects)")
print("  - Fast (called multiple times)")
print("  - Deterministic (same input -> same output)")

***Figure 10.32:** Key functions are called multiple times. Keep them pure and fast.*

### Pitfall 5: Incorrect Comparison

**Listing 10.33 — Comparison Consistency**

In [ ]:
from functools import cmp_to_key

# WRONG: Inconsistent comparison
def bad_compare(a, b):
    # This violates transitivity!
    import random
    return random.choice([-1, 0, 1])

# This will give unpredictable results
# sorted([3, 1, 2], key=cmp_to_key(bad_compare))

# RIGHT: Consistent comparison
def good_compare(a, b):
    if a < b:
        return -1
    elif a > b:
        return 1
    return 0

arr = [3, 1, 2]
print(f"With consistent compare: {sorted(arr, key=cmp_to_key(good_compare))}")

# Comparison must satisfy:
print("\nComparison requirements:")
print("  1. Antisymmetric: if a < b then b > a")
print("  2. Transitive: if a < b and b < c then a < c")
print("  3. Reflexive equality: a == a")

***Figure 10.33:** Custom comparators must be consistent—antisymmetric, transitive, reflexive.*

---
# 📝 Exercises


### Exercise 1: Sort Colors (Dutch National Flag)  (⭐⭐ Medium)

Given an array with n objects colored red, white, or blue (represented as 0, 1, 2), sort them in-place in one pass.

**Expected:** (Expected: [2,0,2,1,1,0] → [0,0,1,1,2,2])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Three Pointers:**

                        Use `low` (boundary for 0s), `mid` (current), `high` (boundary for 2s).
- **Hint 2 - Swap Logic:**

                        If mid sees 0 → swap with low, increment both. If 2 → swap with high, decrement high only.
- **Hint 3 - Mid Movement:**

                        Only increment mid when value is 0 or 1 (not when swapping with high, since new value unknown).
</details>

In [ ]:
# ✏️ [EX1]
# Sort Colors (Dutch National Flag) - One pass

def sort_colors(nums):
    # Your code here
    pass

# Test your implementation (uncomment)
# nums = [2, 0, 2, 1, 1, 0]
# sort_colors(nums)
# print(nums)  # [0, 0, 1, 1, 2, 2]

### Exercise 2: Sort Array by Parity  (⭐ Easy)

Given an integer array, move all even integers to the beginning, followed by all odd integers. Any order within even/odd groups is acceptable.

**Expected:** (Expected: [3,1,2,4] → [2,4,3,1] or similar)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Two Pointers:**

                        Use left pointer starting at 0, right pointer at end.
- **Hint 2 - Movement:**

                        Move left right while pointing to even. Move right left while pointing to odd.
- **Hint 3 - Swap:**

                        When left is odd and right is even, swap them.
</details>

In [ ]:
# ✏️ [EX2]
# Sort Array by Parity - Evens first, odds last

def sort_array_by_parity(nums):
    # Your code here
    pass

# Test your implementation (uncomment)
# nums = [3, 1, 2, 4]
# result = sort_array_by_parity(nums)
# print(result)  # [2, 4, 3, 1] or [4, 2, 1, 3] etc.

### Exercise 3: Relative Sort Array  (⭐⭐ Medium)

Sort arr1 such that elements in arr2 appear first (in arr2's order), followed by remaining elements in ascending order.

**Expected:** (Expected: Custom ordering based on arr2)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Order Map:**

                        Create dict mapping arr2 elements to their indices (0, 1, 2, ...).
- **Hint 2 - Custom Key:**

                        Elements in arr2 sort by their index. Elements not in arr2 sort after.
- **Hint 3 - Fallback:**

                        For elements not in arr2, use (len(arr2), value) as key to sort after arr2 elements by value.
</details>

In [ ]:
# ✏️ [EX3]
# Relative Sort Array - Custom ordering

def relative_sort_array(arr1, arr2):
    # Your code here
    pass

# Test your implementation (uncomment)
# arr1 = [2,3,1,3,2,4,6,7,9,2,19]
# arr2 = [2,1,4,3,9,6]
# print(relative_sort_array(arr1, arr2))  # [2,2,2,1,4,3,3,9,6,7,19]

### Exercise 4: Wiggle Sort II  (⭐⭐⭐ Hard)

Reorder array such that nums[0] < nums[1] > nums[2] < nums[3]... (strict inequality). Assume valid answer always exists.

**Expected:** (Expected: [1,5,1,1,6,4] → [1,6,1,5,1,4])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sort First:**

                        Sort the array to separate smaller and larger values.
- **Hint 2 - Split Halves:**

                        Smaller half goes to even indices (0,2,4...), larger half to odd indices (1,3,5...).
- **Hint 3 - Reverse Placement:**

                        Place from END of each half to avoid adjacent duplicates.
</details>

In [ ]:
# ✏️ [EX4]
# Wiggle Sort II - Strict alternating pattern

def wiggle_sort(nums):
    # Your code here
    pass

# Test your implementation (uncomment)
# nums = [1, 5, 1, 1, 6, 4]
# wiggle_sort(nums)
# print(nums)  # e.g., [1, 6, 1, 5, 1, 4]

### Exercise 5: Merge Sorted Arrays  (⭐⭐ Medium)

Given two sorted arrays nums1 (with extra space) and nums2, merge nums2 into nums1 in-place maintaining sorted order.

**Expected:** (Expected: [1,2,3,0,0,0] + [2,5,6] → [1,2,2,3,5,6])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Start from End:**

                        Three pointers: p1 at end of nums1 data (m-1), p2 at end of nums2 (n-1), p at end of nums1 (m+n-1).
- **Hint 2 - Compare and Place:**

                        Compare nums1[p1] and nums2[p2], place larger at position p, decrement appropriate pointer.
- **Hint 3 - Handle Remaining:**

                        If nums2 has remaining elements after nums1 exhausted, copy them to nums1.
</details>

In [ ]:
# ✏️ [EX5]
# Merge Sorted Arrays - In-place merge

def merge(nums1, m, nums2, n):
    # Your code here
    pass

# Test your implementation (uncomment)
# nums1 = [1,2,3,0,0,0]
# m = 3
# nums2 = [2,5,6]
# n = 3
# merge(nums1, m, nums2, n)
# print(nums1)  # [1,2,2,3,5,6]

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request

WEEK = "Week_10"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Extract exercise answers from IPython history ──
_answers = {}
try:
    _ipy = get_ipython()
    _hist = _ipy.history_manager.get_range(output=False)
    for _sess, _line, _src in _hist:
        _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
        if _m:
            _ex_id = "ex" + _m.group(1)
            _lines = _src.split("\n")
            _clean = "\n".join(_lines[1:]).strip()
            _answers[_ex_id] = {
                "code": _clean,
                "modified": len(_clean) > 5
            }
except Exception:
    pass

# ── Fallback: also check In[] from current session ──
if not _answers:
    try:
        for _src in In:
            if not _src:
                continue
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }
    except NameError:
        pass

# ── Fallback: try reading notebook file (VS Code) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), "r", encoding="utf-8") as _f:
            _nb = json.load(_f)
        for _cell in _nb["cells"]:
            if _cell["cell_type"] != "code":
                continue
            _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure you RAN all exercise cells before submitting.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
